In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold,train_test_split

In [2]:
df = pd.read_csv(r"C:\Users\Parth\Downloads\datasets\USA_Housing.csv")
df

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05
...,...,...,...,...,...,...
4995,60567.94414,7.830362,6.137356,3.46,22837.36103,1.060194e+06
4996,78491.27543,6.999135,6.576763,4.02,25616.11549,1.482618e+06
4997,63390.68689,7.250591,4.805081,2.13,33266.14549,1.030730e+06
4998,68001.33124,5.534388,7.130144,5.44,42625.62016,1.198657e+06


In [3]:
X = df.drop("Price",axis=1).values
Y = df["Price"].values.reshape(-1,1)
n, m = X.shape
print(n,m)

5000 5


In [4]:
kf = KFold(n_splits = 5, shuffle = True, random_state=42)

best_beta = None
best_r2 = -np.inf

beta_values = []
pred_values = []
r2_values = []
scaler_values = []

def add_intercept(X):
    n = X.shape[0]
    return np.hstack([np.ones((n, 1)), X]) 
    
for i,(train_idx,test_idx) in enumerate(kf.split(X),start=1):
    X_train = X[train_idx]
    X_test = X[test_idx]
    Y_train = Y[train_idx]
    Y_test = Y[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    # print(X_train_scaled)    

    X_train_int = add_intercept(X_train_scaled)
    X_test_int = add_intercept(X_test_scaled)

    beta = np.linalg.pinv(X_train_int) @ Y_train
    Y_pred = X_test_int @ beta

    r2 = r2_score(Y_test,Y_pred)

    beta_values.append(beta)
    pred_values.append(Y_pred)
    r2_values.append(r2)
    scaler_values.append(scaler)
    
    print(f"\nFold {i}:")
    print(f"Train samples: {len(train_idx)}, Test samples: {len(test_idx)}")
    print(f"R2 on test: {r2}")
    print(f"Beta on test: {beta}")


Fold 1:
Train samples: 4000, Test samples: 1000
R2 on test: 0.9160772178427468
Beta on test: [[1229576.99256303]
 [ 231741.87665386]
 [ 163580.77656518]
 [ 120724.77138501]
 [   2992.44913605]
 [ 152235.90009854]]

Fold 2:
Train samples: 4000, Test samples: 1000
R2 on test: 0.9143333969899391
Beta on test: [[1230849.57086595]
 [ 228974.55047422]
 [ 165649.73061314]
 [ 121581.22259031]
 [   2092.96829553]
 [ 150487.30339964]]

Fold 3:
Train samples: 4000, Test samples: 1000
R2 on test: 0.9117030618560986
Beta on test: [[1232873.37426632]
 [ 230188.68965355]
 [ 163701.4186032 ]
 [ 120701.00765817]
 [   1252.46057779]
 [ 151647.34958184]]

Fold 4:
Train samples: 4000, Test samples: 1000
R2 on test: 0.9183132603899509
Beta on test: [[1234421.84913666]
 [ 228971.0275827 ]
 [ 163502.2605837 ]
 [ 121992.89112759]
 [   3058.69562741]
 [ 149274.71505081]]

Fold 5:
Train samples: 4000, Test samples: 1000
R2 on test: 0.924478954973583
Beta on test: [[1.23264148e+06]
 [2.29893924e+05]
 [1.6457509

In [5]:
best_fold_idx = int(np.argmax(r2_values))
best_beta = beta_values[best_fold_idx]
best_scaler = scaler_values[best_fold_idx]
print(f"Best fold: fold {best_fold_idx+1} with R2 = {r2_values[best_fold_idx]:.6f}")
print(f"Best beta: {beta_values[best_fold_idx]}")

Best fold: fold 5 with R2 = 0.924479
Best beta: [[1.23264148e+06]
 [2.29893924e+05]
 [1.64575094e+05]
 [1.21797211e+05]
 [7.86135883e+02]
 [1.50562056e+05]]


In [6]:
X_train70, X_test30, y_train70, y_test30 = train_test_split(
    X, Y, test_size=0.30, random_state=123
)

X_train70_scaled = best_scaler.transform(X_train70)
X_test30_scaled = best_scaler.transform(X_test30)

X_train70_int = add_intercept(X_train70_scaled)
X_test30_int  = add_intercept(X_test30_scaled)

y_train70_pred = X_train70_int @ best_beta
y_test30_pred = X_test30_int @ best_beta

r2_train70 = r2_score(y_train70,y_train70_pred)
r2_test30 = r2_score(y_test30,y_test30_pred)

print("Evaluation of best fold's beta on a 70/30 split (using best fold's scaler):")
print(f"R2 on 70% (train) using best_beta: {r2_train70}")
print(f"R2 on 30% (test)  using best_beta: {r2_test30}")

Evaluation of best fold's beta on a 70/30 split (using best fold's scaler):
R2 on 70% (train) using best_beta: 0.9169451291393231
R2 on 30% (test)  using best_beta: 0.919309110301187


In [25]:
def add_bias(X):
    return np.hstack([np.ones((X.shape[0], 1)), X])

def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1.0 - ss_res / ss_tot if ss_tot != 0 else 0.0

def gradient_descent(X, y, lr=0.01, n_iters=1000, verbose=False):
    m, n = X.shape
    theta = np.zeros(n)  # initialize
    losses = []
    for it in range(n_iters):
        preds = X.dot(theta)
        error = preds - y
        loss = (1/(2*m)) * np.sum(error**2)
        losses.append(loss)
        grad = (1/m) * X.T.dot(error)   # shape (n,)
        theta -= lr * grad

        # Divergence safety: if loss explodes, stop early
        if np.isnan(loss) or np.isinf(loss) or loss > 1e20:
            if verbose:
                print(f"Stopped early at iter {it} due to divergence (loss={loss}).")
            break
    return theta, losses

def run_experiment(
    X, y,
    test_size=0.30, val_size=0.14,
    learning_rates=[0.001, 0.01, 0.1, 1],
    n_iters=1000,
    random_state=42,
    stratify=None  # set to y for classification-like stratify (if applicable)
):
    # 1) Split dataset into train/val/test (56/14/30)
    # We'll first split off test (30%), then split remaining 70% into 80/20 -> 56/14
    X_rem, X_test, y_rem, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=True, stratify=stratify
    )
    # proportion of val relative to remaining: val_size / (1 - test_size)
    rel_val_size = val_size / (1 - test_size)
    X_train, X_val, y_train, y_val = train_test_split(
        X_rem, y_rem, test_size=rel_val_size, random_state=random_state, shuffle=True,
        stratify=(y_rem if stratify is not None else None)
    )

    # 2) Feature scaling (fit on train only)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled   = scaler.transform(X_val)
    X_test_scaled  = scaler.transform(X_test)

    # Add bias
    X_train_b = add_bias(X_train_scaled)
    X_val_b   = add_bias(X_val_scaled)
    X_test_b  = add_bias(X_test_scaled)

    results = []
    for lr in learning_rates:
        theta, losses = gradient_descent(X_train_b, y_train, lr=lr, n_iters=n_iters)
        # Predictions
        y_val_pred = X_val_b.dot(theta)
        y_test_pred = X_test_b.dot(theta)
        r2_val = r2_score(y_val, y_val_pred)
        r2_test = r2_score(y_test, y_test_pred)
        results.append({
            'lr': lr,
            'theta': theta,
            'loss_history': losses,
            'final_train_loss': losses[-1] if len(losses)>0 else None,
            'r2_val': r2_val,
            'r2_test': r2_test,
            'diverged': (len(losses) < n_iters)
        })
        print(f"lr={lr:>6} | val_R2={r2_val: .6f} | test_R2={r2_test: .6f} | final_loss={losses[-1]:.6e} | diverged={results[-1]['diverged']}")

    # Choose best by validation R^2 (higher is better)
    best = max(results, key=lambda r: r['r2_val'])
    print("\nBest model (by validation R²):")
    print(f"  lr = {best['lr']}")
    print(f"  val R² = {best['r2_val']:.6f}")
    print(f"  test R² = {best['r2_test']:.6f}")
    print(f"  theta (bias first) = {best['theta']}")
    return {
        'results': results,
        'best': best,
        'splits': {
            'X_train': X_train, 'X_val': X_val, 'X_test': X_test,
            'y_train': y_train, 'y_val': y_val, 'y_test': y_test
        },
        'scaler': scaler
    }

  
# best = res['best']
# plt.plot(best['loss_history'])
# plt.xlabel("Iteration")
# plt.ylabel("Train loss (1/(2m) sum_sq)")
# plt.title(f"Loss history (lr={best['lr']})")
# plt.show()


In [27]:
feature_cols=["Avg. Area Income","Avg. Area House Age","Avg. Area Number of Rooms","Avg. Area Number of Bedrooms","Area Population"]
target_col=["Price"]
X_all = df[feature_cols].values
y_all = df[target_col].values.squeeze()

res = run_experiment(X_all, y_all, learning_rates=[0.001,0.01,0.1,1], n_iters=1000)

lr= 0.001 | val_R2=-0.800529 | test_R2=-0.987306 | final_loss=1.160438e+11 | diverged=False
lr=  0.01 | val_R2= 0.909850 | test_R2= 0.914744 | final_loss=5.037731e+09 | diverged=False
lr=   0.1 | val_R2= 0.909831 | test_R2= 0.914757 | final_loss=5.037687e+09 | diverged=False
lr=     1 | val_R2= 0.909831 | test_R2= 0.914757 | final_loss=5.037687e+09 | diverged=False

Best model (by validation R²):
  lr = 0.01
  val R² = 0.909850
  test R² = 0.914744
  theta (bias first) = [1232451.13728891  234580.02202884  162424.51400554  121501.08421757
    3090.26111211  151605.17031522]
